In [147]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수에서 GPT API 키 가져오기
google_key = os.getenv("GOOGLE_API_KEY")

In [148]:
genai.configure(api_key=google_key)
model = genai.GenerativeModel('gemini-1.5-flash') 

In [149]:
import os
google_key = os.getenv("GOOGLE_API_KEY")
if not google_key:
    raise ValueError("Google API 키가 설정되지 않았습니다.")
genai.configure(api_key=google_key)

In [150]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [45]:
import json
import os
import pandas as pd

path = "chat_history_gemini.json"

# # CSV 파일 경로
# csv_file = "경로설정 고고링"

# # CSV 데이터 로드
# def load_csv_data(file_path):
#     try:
#         return pd.read_csv(file_path)
#     except FileNotFoundError:
#         print(f"Error: {file_path} 파일을 찾을 수 없습니다.")
#         return pd.DataFrame()

# # CSV 데이터 로드
# csv_data = load_csv_data(csv_file)


# 대화 기록 로드 함수
def load_history():
    if os.path.exists(path):
        with open(path, "r", encoding = "utf-8") as file:
            return json.load(file)
    return [{"role" : "system", "content" : "당신은 친절한 화장품 안내 가이드입니다"}]

# 대화 기록 저장 함수
def save_history(messages):
    with open(path, "w", encoding = "utf-8",) as file:
        json.dump(messages, file, ensure_ascii = False, indent = 4)


# gemini 모델 호출 함수
def get_completion_from_messages(messages, temperature = 1):
    response = client.chat.completions.create(
        model = "gemini-1.5-flash",
        messages = messages,
        temperature = temperature
    )
    return response.choices[0].message.content

In [5]:
import google.generativeai as genai
import json
import os

# Google Gemini API 설정
genai.configure(api_key=google_key)

# 시스템 메시지 설정
system_instruction = """당신은 친절한 화장품 안내 가이드 챗봇명 피카추야 
너의 이름은 피카츄가 아닌 피카추야
질문자가 원하는 종류의 화장품을 추천해줘."""

model = genai.GenerativeModel('gemini-1.5-flash', system_instruction=system_instruction)

# JSON 파일 경로
history_file = "chat_history_gemini.json"



# 대화 기록 로드 함수
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r", encoding="utf-8") as file:
            return json.load(file)
    # 파일이 없으면 기본 시스템 메시지 반환
    return [{"role": "user", "parts": [system_instruction]}]

# 대화 기록 저장 함수 (새로운 대화 내용 추가)
def save_history(messages):
    if os.path.exists(history_file):
        with open(history_file, "r", encoding="utf-8") as file:
            existing_messages = json.load(file)
    else:
        existing_messages = []

    # 기존 메시지와 새로운 메시지 병합
    updated_messages = existing_messages + messages

# JSON 파일에 대화 기록 저장
def save_history(messages, file_path=history_file):
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(messages, file, ensure_ascii=False, indent=4)

# 대화 시작
messages = load_history()
print("챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요")

while True:
    user_input = input("사용자: ")
    if user_input.lower() in ["exit", "quit", "종료", "나가기"]:
        print("챗봇 : 대화를 종료합니다.")
        save_history(messages)
        break

    # 사용자 입력 추가
    messages.append({"role": "user", "parts": [user_input]})

    # 화장품 종류 묻기
    print("챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 알려주세요")
    skin_input = input("사용자: ")
    if skin_input.lower() in ["exit", "quit", "종료", "나가기"]:
        print("챗봇 : 대화를 종료합니다.")
        save_history(messages)
        break

    # 사용자 입력 저장
    messages.append({"role": "user", "parts": [skin_input]})

    # AI 응답 생성
    response = model.generate_content(messages)

    # 응답 추가
    messages.append({"role": "assistant", "parts": [response.text]})

    # 대화 출력
    print(f"챗봇(피카추) : {response.text}")

    # 대화 기록을 파일에 저장
    save_history(messages)


챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요
챗봇 : 대화를 종료합니다.


챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요
챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 알려주세요
챗봇(피카추) : 피카추야입니다! 지성 피부인데 보습 고민이시라면,  세럼 선택이 중요해요!  ✨  지성 피부는 피지 분비가 많아 번들거리지만,  피지가 많다고 해서 수분까지 충분하다는 건 아니에요.  수분 부족은 피지 분비를 더욱 촉진시켜  트러블을 악화시킬 수 있답니다.  그래서 지성 피부 보습은  **유분감 없이 수분을 충분히 공급**하는 것이  핵심입니다!


**추천 세럼 종류 및 성분:**

* **젤 타입 세럼:**  가볍고 산뜻한 젤 타입 세럼이 지성 피부에 적합해요.  끈적임 없이 빠르게 흡수되어  수분을 공급하고 산뜻한 마무리감을 제공합니다.

* **히알루론산 세럼:**  히알루론산은 수분을 끌어당기는 능력이 뛰어나  피부에 풍부한 수분을 공급해 줍니다.  하지만  너무 고농축 제형은 끈적일 수 있으니  적절한 농도의 제품을 선택하는 것이 중요해요.

* **판테놀 (비타민 B5) 세럼:**  피부 진정 및 재생 효과가 뛰어나고,  피부 장벽 강화에도 도움을 줍니다.  건조하고 자극받은 피부를 진정시키고  수분을 공급하여  피부 컨디션을 개선하는 데 효과적입니다.

* **니아시나마이드 세럼:**  피지 조절과  피부 장벽 강화에 도움을 주는 성분으로,  과도한 피지 분비를 조절하고  수분 밸런스를 맞춰주는 데 효과적입니다.  하지만  처음 사용할 때는  피부 자극을 확인하며  소량씩 사용해 보는 것이 좋습니다.


**피해야 할 성분:**

* **과도한 유분:**  미네랄 오일, 실리콘 오일 등 과도한 유분감을 주는 성분은 모공을 막고 트러블을 유발할 수 있으므로 피하세요.
* **코메도제닉 성분:**  모공을 막는 성분(코메도제닉 성분)이 함유된 세럼은 피하는 것이 좋습니다.  제품 설명을 꼼꼼하게 확인하세요.
* **알코올:**  알코올 성분은  피부를 건조하게 만들 수 있으므로  알코올 함량이 높은 제품은 피하는 것이 좋습니다.


**추가 팁:**

* 세럼은 토너 다음 단계에 사용하고,  가벼운 크림이나 에멀전으로 마무리하여 수분을 잠가주세요.
* 손바닥에 적당량을 덜어 얼굴 전체에 부드럽게 펴 바르고,  가볍게 두드려 흡수시켜주세요.
* 꾸준히 사용하는 것이 중요해요!


제품을 선택하실 때는  성분표를 꼼꼼하게 확인하시고,  피부에 자극이 되지 않는지  패치 테스트를 해보시는 것을 추천드려요.  더 궁금한 점이 있다면 언제든지 저에게 물어보세요! 피카츄야가 도와드릴게요! 💖

챗봇 : 대화를 종료합니다.
챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요
챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 입력해주세요 (예: 건성, 주름)
챗봇(피카추) : 입력 형식이 올바르지 않습니다. 예: 건성, 주름
챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 입력해주세요 (예: 건성, 주름)
챗봇(피카추) : 안녕하세요! 피카추야입니다!  건성 피부에 주름 고민이시라니, 꼼꼼하게 관리해 드릴게요!  건성 피부는 수분 부족으로 인해 주름이 더욱 도드라져 보일 수 있으니,  수분 공급과 함께 주름 개선에 효과적인 성분이 함유된 선케어 제품을 추천해 드릴게요.


**추천 선케어 제품 (타입별):**

**1. 고보습, 안티에이징 기능성 선크림:**

* **특징:** 건성 피부에 꼭 필요한 풍부한 수분감과 함께 주름 개선, 피부 탄력 개선에 도움을 주는 기능성 성분이 함유된 제품. 백탁 현상이 적고, 자외선 차단 지수(SPF 50+ PA++++ 이상)가 높은 제품을 선택하는 것이 좋습니다.
* **추천 성분:** 히알루론산, 세라마이드, 콜라겐, 레티놀(저녁용), 나이아신아마이드, 비타민 C 유도체 등.  단, 레티놀은 자외선에 취약하므로, 저녁에 사용하는 것이 좋습니다.
* **제형:** 크림, 로션 타입.  오일 성분이 함유된 리치한 제형을 추천합니다.
* **주의사항:**  제품 사용 전 패치 테스트를 해보는 것이 좋습니다.  레티놀 성분이 포함된 제품은 자외선 차단제와 함께 사용해야 합니다.

**2. 톤업 선크림 (수분감 있는 제형):**

* **특징:** 자외선 차단 기능과 함께 피부 톤을 밝혀주는 효과가 있어 화사한 피부 표현을 원하시는 분들에게 적합합니다.  하지만, 톤업 기능이 있는 제품은 피부에 자극을 줄 수 있으므로, 건성 피부이신 경우 수분감이 충분한 제품을 선택하는 것이 중요합니다.
* **추천 성분:**  히알루론산, 세라마이드와 같이 수분을 공급하는 성분이 함유된 제품이 좋습니다.
* **제형:** 수분 크림, 에센스 타입.


**추천 브랜드 (예시):** *(구체적인 제품명은 직접 확인해주세요.  피부 타입과 취향에 따라 선택하는 것이 가장 중요합니다!)*

* **고가 브랜드:**  Estee Lauder, SK-II, Sulwhasoo 등 (안티에이징 기능이 강화된 제품 많음)
* **중저가 브랜드:**  CeraVe, La Roche-Posay,  Dr.Althea 등 (건성 피부에 좋은 보습 성분 함유 제품 많음)


**제품 선택 팁:**

* **자외선 차단 지수 확인:** SPF 50+ PA++++ 이상 제품을 선택하세요.
* **성분 확인:**  자신의 피부 타입에 맞는 성분이 함유되었는지 확인하고, 알레르기 유발 성분이 없는지 확인하세요.
* **후기 확인:**  다른 사용자들의 리뷰를 참고하여 제품의 사용감과 효과를 확인하세요.
* **테스트 후 구매:**  매장에서 테스트 후 구매하는 것을 추천합니다.


주름 개선은 장기적인 관리가 필요하므로, 선크림과 함께 꾸준한 보습 관리,  주름 개선 기능성 화장품 사용 등을 병행하는 것을 추천드립니다.  더 궁금한 점이 있으면 언제든지 저에게 물어보세요!  피카추!⚡

챗봇 : 대화를 종료합니다.

## csv 튜플 형식

In [ ]:
# 튜플 형식으로 csv 파일 저장
file_tuple = (
    "cream_aesturaatobarrier365cream.csv",
    "cream_drgblacksnailcream.csv",
    "cream_larocheposaycicaplastbaume.csv",
    "cream_redblemishclearsoothingcream.csv",
    "cream_snatureaquasqualanemoisturizingcream.csv",
    "mask_biodancebiocollagenrealdeepmask.csv",
    "mask_medihealmadecassosideblemishpad.csv",
    "mask_medihealteatreeessentialmask.csv",
    "mask_numbuzinno5vitaminspotlightsheetmask.csv",
    "mask_torridendiveinlowmolecularhyaluronicacidmask.csv",
    "serum_cosrxthevitaminc23serum.csv",
    "serum_greentangerinevitacdarkspotcareserum.csv",
    "serum_ioperetinolsuperbounceserum.csv",
    "serum_numbuzin3skinserum.csv",
    "serum_torridendiveinserum.csv",
    "suncare_ahcmastersairrichsunstick.csv",
    "suncare_atheveganreliefsuncream.csv",
    "suncare_plantnarawaterproofsunspray.csv",
    "suncare_dalbapinktoneupsuncream.csv",
    "suncare_roundlabbirchmoisturesuncream.csv"
)


In [10]:
pip show streamlit

Name: streamlit
Version: 1.40.2
Summary: A faster way to build and share data apps
Home-page: https://streamlit.io
Author: Snowflake Inc
Author-email: hello@streamlit.io
License: Apache License 2.0
Location: c:\Users\RMARKET\anaconda3\envs\langchain\Lib\site-packages
Requires: altair, blinker, cachetools, click, gitpython, numpy, packaging, pandas, pillow, protobuf, pyarrow, pydeck, requests, rich, tenacity, toml, tornado, typing-extensions, watchdog
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [6]:
import google.generativeai as genai
import pandas as pd
import json
import os

# Google Gemini API 설정
genai.configure(api_key=google_key)

# 시스템 메시지 설정
# csv 파일을 읽어오는 프롬프트 작성
system_instruction = """당신은 친절한 화장품 안내 가이드 챗봇명 피카추야 
너의 이름은 피카츄가 아닌 피카추야
질문자가 원하는 종류의 화장품을 추천해줘."""

model = genai.GenerativeModel('gemini-1.5-flash', system_instruction=system_instruction)

# 모든 파일을 반복하며 읽기
# 변수명 변경
# for file in file_tuple:
#     data = pd.read_csv(file)
#     print(f"{file}의 데이터:\n", data.head())

# JSON 파일 경로
history_file = "test_history.json"
csv_file = "cream_aesturaatobarrier365cream.csv"

# CSV 데이터 로드 함수
def load_csv_data(file_path):
    try:
        return pd.read_csv(file_path, header=None, names=["리뷰"])
    except FileNotFoundError:
        print(f"Error: {file_path} 파일을 찾을 수 없습니다.")
        return pd.DataFrame()
    
    

# CSV 데이터 로드
csv_data = load_csv_data(csv_file)

# 키워드를 기반으로 리뷰 검색
def search_reviews(keywords):
    if csv_data.empty:
        return "CSV 데이터가 없습니다. 데이터를 확인해주세요."

    # 키워드가 포함된 리뷰 검색
    matched_reviews = csv_data[csv_data["리뷰"].str.contains('|'.join(keywords), na=False)]
    if matched_reviews.empty:
        return "해당 키워드에 맞는 리뷰를 찾을 수 없습니다."
    return matched_reviews["리뷰"].to_string(index=False)

# 대화 기록 로드 함수
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r", encoding="utf-8") as file:
            return json.load(file)
    return [{"role": "user", "parts": [system_instruction]}]

# JSON 파일에 대화 기록 저장
def save_history(messages, file_path=history_file):
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(messages, file, ensure_ascii=False, indent=4)

# 대화 시작
messages = load_history()
print("챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요")

while True:
    user_input = input("사용자: ")
    if user_input.lower() in ["exit", "quit", "종료", "나가기"]:
        print("챗봇 : 대화를 종료합니다.")
        save_history(messages)
        break

    messages.append({"role": "user", "parts": [user_input]})

    print("챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 입력해주세요 (예: 건성, 주름)")
    skin_input = input("사용자: ")

    # 입력 유효성 검사
    if skin_input.lower() in ["exit", "quit", "종료", "나가기"]:
        print("챗봇 : 대화를 종료합니다.")
        save_history(messages)
        break

    try:
        skin_type, concern = map(str.strip, skin_input.split(','))
    except ValueError:
        print("챗봇(피카추) : 입력 형식이 올바르지 않습니다. 예: 건성, 주름")
        continue

    # 키워드 기반 리뷰 검색
    csv_result = search_reviews([skin_type, concern])

    # 모델에 검색 결과 전달
    if "리뷰를 찾을 수 없습니다" in csv_result:
        model_input = f"피부 타입: {skin_type}, 고민: {concern}. {csv_result}"
    else:
        model_input = f"피부 타입: {skin_type}, 고민: {concern}. 관련 리뷰:\n{csv_result}"

    messages.append({"role": "user", "parts": [model_input]})

    # AI 응답 생성
    response = model.generate_content(messages)

    messages.append({"role": "assistant", "parts": [response.text]})
    print(f"챗봇(피카추) : {response.text}")

    save_history(messages)


챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요
챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 입력해주세요 (예: 건성, 주름)
챗봇(피카추) : 안녕하세요, 피카추야입니다! 지성피부에 주름이 고민이시라니,  피부 타입에 맞는 선케어가 중요하겠네요!  지성 피부는 과도한 피지 분비로 인해 번들거림이 심하고,  모공이 막히기 쉬워 트러블이 발생하기 쉽습니다.  주름 고민까지 있다면,  가볍고 산뜻하면서도 자외선 차단 효과가 뛰어난 제품을 선택해야 합니다.  단순히 번들거림만 방지하는 것이 아니라,  피부에 자극을 주지 않고 꼼꼼하게 자외선을 차단하는 것이 중요해요!


**지성 피부 & 주름 고민을 위한 선케어 제품 선택 가이드:**

* **가벼운 제형:**  젤 타입, 에센스 타입, 플루이드 타입 등 가볍고 산뜻한 제형을 선택하세요. 크림이나 로션 타입은 피부에 무겁게 느껴질 수 있습니다.
* **번들거림 방지:**  세범 컨트롤 기능이 있는 제품을 선택하면 번들거림을 예방하는데 도움이 됩니다.
* **자외선 차단 지수:** SPF 50+ PA++++ 이상의 제품을 선택하여 강력한 자외선 차단 효과를 얻으세요.
* **무기자차 or 유기자차:**  무기자차는 백탁 현상이 있을 수 있지만 자극이 적고, 유기자차는 백탁 현상이 적지만 자극이 있을 수 있습니다.  자신의 피부 타입에 맞는 것을 선택하세요.  민감한 피부라면 무기자차를 추천합니다.
* **항산화 성분:** 비타민 C 유도체, 비타민 E 등의 항산화 성분이 함유된 제품을 선택하면 광노화를 예방하는데 도움이 됩니다.
* **저자극 포뮬러:**  피부 트러블을 유발할 수 있는 향료, 색소, 파라벤 등의 성분이 없는 저자극 제품을 선택하세요.


**추천 브랜드 및 성분 (예시):** *(구체적인 제품명은 직접 확인해주세요.  피부 타입과 취향에 따라 선택하는 것이 가장 중요합니다!)*

* 

챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요
챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 알려주세요
챗봇(피카추) : 피카추야입니다! 지성 피부인데 보습 고민이시라면,  세럼 선택이 중요해요!  ✨  지성 피부는 피지 분비가 많아 번들거리지만,  피지가 많다고 해서 수분까지 충분하다는 건 아니에요.  수분 부족은 피지 분비를 더욱 촉진시켜  트러블을 악화시킬 수 있답니다.  그래서 지성 피부 보습은  **유분감 없이 수분을 충분히 공급**하는 것이  핵심입니다!


**추천 세럼 종류 및 성분:**

* **젤 타입 세럼:**  가볍고 산뜻한 젤 타입 세럼이 지성 피부에 적합해요.  끈적임 없이 빠르게 흡수되어  수분을 공급하고 산뜻한 마무리감을 제공합니다.

* **히알루론산 세럼:**  히알루론산은 수분을 끌어당기는 능력이 뛰어나  피부에 풍부한 수분을 공급해 줍니다.  하지만  너무 고농축 제형은 끈적일 수 있으니  적절한 농도의 제품을 선택하는 것이 중요해요.

* **판테놀 (비타민 B5) 세럼:**  피부 진정 및 재생 효과가 뛰어나고,  피부 장벽 강화에도 도움을 줍니다.  건조하고 자극받은 피부를 진정시키고  수분을 공급하여  피부 컨디션을 개선하는 데 효과적입니다.

* **니아시나마이드 세럼:**  피지 조절과  피부 장벽 강화에 도움을 주는 성분으로,  과도한 피지 분비를 조절하고  수분 밸런스를 맞춰주는 데 효과적입니다.  하지만  처음 사용할 때는  피부 자극을 확인하며  소량씩 사용해 보는 것이 좋습니다.


**피해야 할 성분:**

* **과도한 유분:**  미네랄 오일, 실리콘 오일 등 과도한 유분감을 주는 성분은 모공을 막고 트러블을 유발할 수 있으므로 피하세요.
* **코메도제닉 성분:**  모공을 막는 성분(코메도제닉 성분)이 함유된 세럼은 피하는 것이 좋습니다.  제품 설명을 꼼꼼하게 확인하세요.
* **알코올:**  알코올 성분은  피부를 건조하게 만들 수 있으므로  알코올 함량이 높은 제품은 피하는 것이 좋습니다.


**추가 팁:**

* 세럼은 토너 다음 단계에 사용하고,  가벼운 크림이나 에멀전으로 마무리하여 수분을 잠가주세요.
* 손바닥에 적당량을 덜어 얼굴 전체에 부드럽게 펴 바르고,  가볍게 두드려 흡수시켜주세요.
* 꾸준히 사용하는 것이 중요해요!


제품을 선택하실 때는  성분표를 꼼꼼하게 확인하시고,  피부에 자극이 되지 않는지  패치 테스트를 해보시는 것을 추천드려요.  더 궁금한 점이 있다면 언제든지 저에게 물어보세요! 피카츄야가 도와드릴게요! 💖

챗봇 : 대화를 종료합니다.
챗봇(피카추) : 원하시는 화장품 종류(선케어, 크림, 세럼, 마스크)를 알려주세요
챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 입력해주세요 (예: 건성, 주름)
챗봇(피카추) : 안녕하세요! 피카추야입니다! 지성 피부인데 보습이 고민이시라니,  조금 특별한 케어가 필요하시겠네요!  지성 피부는 피지 분비가 많아서 번들거림이 심하지만,  피부 속 수분은 부족할 수 있습니다.  그래서 보습은 중요하지만, 끈적이거나 무거운 제형은 피해야 합니다.  가볍고 산뜻하면서도 깊은 보습을 제공하는 세럼을 추천해 드릴게요!


**지성 피부 보습 세럼 선택 가이드:**

* **가벼운 제형:** 젤, 에센스 타입의 가벼운 제형을 선택하는 것이 좋습니다. 크림이나 오일 타입은 피부에 부담이 될 수 있습니다.
* **수분 공급 성분:** 히알루론산, 판테놀, 글리세린 등 수분을 머금는 성분이 함유된 제품을 선택하세요.
* **피지 조절 성분:**  피지 분비를 조절하는 성분 (예: 살리실산, 티트리 오일 등)이 함유된 제품이라면 더욱 좋습니다. 하지만 민감성 피부라면 피지 조절 성분이 자극을 줄 수 있으므로 주의하세요.  피부 테스트 후 사용하는 것을 권장합니다.
* **흡수력:** 빠르게 흡수되는 제품을 선택하세요. 끈적임 없이 산뜻하게 마무리되는 제품이 좋습니다.
* **무향료, 저자극:**  향료나 자극적인 성분이 들어있지 않은 제품을 선택하여 피부 트러블을 예방하세요.


**추천 세럼 성분 (예시):**

* **히알루론산:**  피부에 수분을 공급하고 유지하는 데 탁월한 효과를 가진 성분입니다.
* **판테놀:**  피부 진정 및 보습 효과가 뛰어나며, 피부 장벽 강화에도 도움이 됩니다.
* **글리세린:**  피부 표면에 수분막을 형성하여 수분 증발을 막아줍니다.
* **세라마이드:**  피부 장벽을 강화하여 수분 손실을 예방하는 데 효과적입니다.


**추천 브랜드 (예시):** *(구체적인 제품명은 직접 확인해주세요.  피부 타입과 취향에 따라 선택하는 것이 가장 중요합니다!)*

* **중저가 브랜드:**  The Ordinary (히알루론산 세럼),  Isntree (히알루론산 세럼),  Cosrx (피지 조절 세럼)  등 (가성비 좋은 제품 많음)
* **고가 브랜드:**  Estée Lauder, SK-II 등 (고기능성 제품 많음)


**추가 팁:**

* 세럼을 바르기 전에 토너로 피부결을 정돈하고, 수분 크림으로 마무리하여 보습 효과를 높여주세요.
* 아침, 저녁으로 꾸준히 사용하는 것이 좋습니다.
* 제품 사용 전에 패치 테스트를 해보는 것이 좋습니다.


지성 피부 보습은 쉽지 않지만,  적절한 제품을 선택하고 꾸준히 관리하면 충분히 개선될 수 있습니다.  피카추가 응원할게요!  다른 궁금한 점이 있으면 언제든지 물어보세요!  피카추!⚡

챗봇(피카추) : 피부 타입(건성, 지성, 복합성, 민감성)과 피부 고민(주름 고민, 보습 고민, 미백 고민, 진정 고민)을 입력해주세요 (예: 건성, 주름)
챗봇 : 대화를 종료합니다.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import CSVLoader

try:
    # CSVLoader로 문서 로드
    loader = CSVLoader(file_path="data/total_reviews.csv")
    documents = loader.load()

    # 텍스트 분할
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(documents)

    # FAISS 벡터스토어 생성
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(split_docs, embeddings)
    print("FAISS 벡터스토어 생성 완료.")
except Exception as e:
    print(f"벡터스토어 생성 중 오류 발생: {e}")


In [ ]:
# 벡터스토어 생성 후 관련 키워드 검색
query = "건성 피부에 적합한 보습 크림 추천"
results = vectorstore.similarity_search(query, k=5)  # k는 반환할 문서 개수

# 결과 출력
for idx, result in enumerate(results):
    print(f"결과 {idx + 1}: {result.page_content}")


결과 1: ﻿불용어 제거 리뷰: 한달이상 후기 겨울에 사용했던 크림이에요 건성피부가 겨울에 사용하기엔 보습력이 약한편이지만 자기전에 앰플 먼저 바르고 수분크림 듬뿍 올려서 수분 보충용으로 사용했더니 괜찮더라구요 건성피부는 겨울 여름에 사용하기 좋을 제품이에요 향은 없고 가벼운 제형이에요
키워드: 한달이상, 후기
제품명: 닥터지 레드 블레미쉬 클리어 수딩 크림
피부타입: 복합성, 건성, 지성
기능: 보습, 진정
제품종류: 크림
결과 2: ﻿불용어 제거 리뷰: 건성의 건성 피부를 가진 1인으로써 만족합니다
키워드: 건성의, 건성
제품명: 에스네이처 아쿠아 스쿠알란 수분크림
피부타입: 복합성, 건성, 지성
기능: 보습, 진정
제품종류: 크림
결과 3: ﻿불용어 제거 리뷰: 내 남자친구가 좋아하는 크림 그는 지성 피부를 가지고 있고 항상 크림이 그의 피부를 지성으로 만들고 발진을 일으킨다고 불평합니다 남자친구는 크림을 사용하기 시작한 이후로 피부의 혼란이 개선되었다고 합니다 두껍고 기름진 제품을 바를 여드름이 잘 생기는 지성 피부를 가진 사람들에게 크림을 추천합니다
키워드: 남자친구가, 좋아하는
제품명: 닥터지 레드 블레미쉬 클리어 수딩 크림
피부타입: 복합성, 건성, 지성
기능: 보습, 진정
제품종류: 크림
결과 4: ﻿불용어 제거 리뷰: 수부지한테는 적합한 크림입니다 얼굴이 여드름이 심했었는데요 2년째쓰는데 얼굴이 엄청나게 나아졌습니다 기적인거죠 얼굴에 맞아서 행복했습니다 이렇개 긴시간동안 제가 많은노력을했는데 꼭 크림은 매일 발라주고 다녔습니다 다들 크림 사세요 ㅜ
키워드: 수부지한테는, 적합한
제품명: 닥터지 레드 블레미쉬 클리어 수딩 크림
피부타입: 복합성, 건성, 지성
기능: 보습, 진정
제품종류: 크림
결과 5: ﻿불용어 제거 리뷰: 구지 제가 설명안해도 너무너무 유명해서 다들 아시지요 지성피부 특화된 크림이라고 생각하시면 될거같아요 피부가 민감하시거나 가벼운 라이트한 크림을 찾으시는분 여름에 바를 크림이 필요하신분 제형이 보면 되게 젤리같은 제형이고 바

In [40]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS

# CSV 파일 로드 및 벡터스토어 생성
csv_file_path = "data/total_reviews.csv"
loader = CSVLoader(file_path=csv_file_path)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = text_splitter.split_documents(documents)

# OpenAI 임베딩 및 FAISS 벡터스토어 생성
embeddings = OpenAIEmbeddings()  # OpenAI 임베딩
vectorstore = FAISS.from_documents(split_docs, embeddings)

# Chat Completion 모델 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0)  # Chat Completion 모델
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 사용자 입력 실행
user_query = "건성 피부에 적합한 보습 크림 추천"
response = qa_chain.run(user_query)

# 결과 출력
print("응답:", response)


응답: 건성 피부에 적합한 보습 크림으로는 "닥터지 레드 블레미쉬 클리어 수딩 크림"과 "에스네이처 아쿠아 스쿠알란 수분크림"을 추천합니다. 두 제품 모두 건성 피부에 보습과 진정 효과가 있는 크림입니다.


In [151]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS

# CSV 파일 로드 및 벡터스토어 생성
csv_file_path = "data/total_reviews.csv"
loader = CSVLoader(file_path=csv_file_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=150)
split_docs = text_splitter.split_documents(documents)

# OpenAI 임베딩 및 FAISS 벡터스토어 생성
embeddings = OpenAIEmbeddings()  # OpenAI 임베딩
vectorstore = FAISS.from_documents(split_docs, embeddings)

# Chat Completion 모델 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0)  # Chat Completion 모델
retriever = vectorstore.as_retriever(search_kwargs={"k": 700}) # 기본값 k=5
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 모든 결과를 반환하도록 수정
# response = qa_chain.run(input=user_query, return_all=True)

# 사용자 입력 실행
user_query = "마스크 종류 개수와 그 제품명을 알려줘. 중복된 제품은 제외해"
response = qa_chain.run(user_query)

# 결과 출력
print("응답:", response)

C:\Users\RMARKET\AppData\Local\Temp\ipykernel_2792\3827709016.py:21: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-4o", temperature=0)  # Chat Completion 모델
C:\Users\RMARKET\AppData\Local\Temp\ipykernel_2792\3827709016.py:30: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = qa_chain.run(user_query)


응답: 마스크 종류는 총 4개입니다. 제품명은 다음과 같습니다:

1. 메디힐 티트리 에센셜 마스크
2. 바이오던스 바이오 콜라겐 리얼 딥 마스크
3. 넘버즈인 5번 글루타치온씨 흔적 필름팩
4. 토리든 다이브인 저분자 히알루론산 마스크


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
# CSV 파일 로드 및 벡터스토어 생성
csv_file_path = "data/total_reviews.csv"
loader = CSVLoader(file_path=csv_file_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
split_docs = text_splitter.split_documents(documents)
# OpenAI 임베딩 및 FAISS 벡터스토어 생성
embeddings = OpenAIEmbeddings()  # OpenAI 임베딩
vectorstore = FAISS.from_documents(split_docs, embeddings)
# Chat Completion 모델 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # Chat Completion 모델
retriever = vectorstore.as_retriever()
# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
# 사용자 입력 실행
user_query = "복합성에 좋은 크림을 모두 추천해줘"
# 모든 결과를 반환하도록 수정
response = qa_chain.run(user_query, return_all=True)
# 결과 출력
print("응답:")
for idx, res in enumerate(response):
    print(f"{idx + 1}. {res}")

## langsmith

In [153]:
pip install streamlit-option-menu

   ---------------------------------------- 0.0/829.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/829.3 kB ? eta -:--:--
   ------------ --------------------------- 262.1/829.3 kB ? eta -:--:--
   ---------------------------------------- 829.3/829.3 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip show langchain

Name: langchain
Version: 0.3.11
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\Users\RMARKET\anaconda3\envs\langchain\Lib\site-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community, langchain-teddynote, ragas
Note: you may need to restart the kernel to use updated packages.


In [166]:
from dotenv import load_dotenv

load_dotenv()

True

In [167]:
from langchain_teddynote import logging
logging.langsmith("PIKACHU")

LangSmith 추적을 시작합니다.
[프로젝트명]
PIKACHU


In [168]:
from langchain_community.document_loaders import PDFPlumberLoader

In [169]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [170]:
model = ChatOpenAI(
    model = "gpt-4o"
)

In [171]:
prompt = ChatPromptTemplate.from_messages(
    # ChatPromptTemplate 대화형 프롬포트
    [
        ("system", "너는 친절한 화장품 안내 가이드야 "),
        MessagesPlaceholder(variable_name = "chat_history"),
        # 이전 대화내용을 chat_history 이름으로 저장할 수 있게 함
        ("human", "{input}")
    ]
)

In [172]:
memory = ConversationBufferMemory(return_messages = True, memory_key = "chat_history")

In [173]:
memory.load_memory_variables({})

{'chat_history': []}

In [174]:
runnable = RunnablePassthrough.assign(
    chat_history = RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
)

In [163]:
chain = runnable | prompt | model

In [165]:
respone = chain.invoke({"input" : "복합성 피부에 맞는 화장품 추천 해줘"})
print(respone.content)

복합성 피부는 일반적으로 T존(이마, 코, 턱)은 기름지고, U존(볼과 턱선)은 건조한 특징을 가지고 있습니다. 따라서 이 두 가지 피부 타입을 균형 있게 관리할 수 있는 제품을 선택하는 것이 중요합니다. 다음은 복합성 피부에 적합한 화장품 추천 목록입니다:

1. **클렌저**:
   - 젤 타입 클렌저: 피부의 유분을 제거하면서도 수분을 유지해줍니다.
   - 폼 클렌저: 과도한 유분을 제거하면서도 피부를 건조하게 만들지 않는 제품을 선택하세요.

2. **토너**:
   - 알코올 프리 토너: 피부를 진정시키고 pH 균형을 맞추어줍니다.
   - 수분 공급이 가능한 토너: 건조한 부분에 수분을 공급하면서도 유분을 조절할 수 있는 제품이 좋습니다.

3. **세럼**:
   - 하이알루론산 세럼: 수분 공급을 통해 건조한 부위를 촉촉하게 유지해줍니다.
   - 비타민 C 세럼: 피부 톤을 고르게 하고 밝게 만들어줍니다.

4. **모이스처라이저**:
   - 젤 크림: 유분이 적고 수분 공급이 뛰어난 제품으로, 복합성 피부에 적합합니다.
   - 유분이 적은 수분 크림: 유분기를 최소화하면서 수분을 충분히 공급해주는 제품을 선택하세요.

5. **선크림**:
   - 무기자차(물리적 자외선 차단제): 민감한 피부에도 적합하며, 유분이 적어 복합성 피부에 잘 맞습니다.

6. **마스크팩**:
   - 수분 공급 마스크: 일주일에 1-2회 사용하여 피부에 충분한 수분을 공급해줍니다.
   - 점토 마스크: T존의 과도한 유분을 제거하는 데 도움을 줍니다.

이 외에도 제품을 선택할 때에는 자신의 피부에 알레르기가 없는지, 그리고 성분이 순한지를 고려하는 것이 중요합니다. 새로운 제품을 사용하기 전에 항상 패치 테스트를 해보는 것을 추천드립니다.


In [128]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bd2da860-aa8b-49f2-901c-4f6082d356f5-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20})

In [130]:
memory.load_memory_variables({})

{'chat_history': []}

In [131]:
memory.save_context({"human" : "만나서 반갑습니다 제 이름은 지수입니다"}, {"ai" : respone.content})

In [132]:
respone = chain.invoke({"input" : "이전에 알려준 정보 다시 알려줘"})
print(respone.content)

물론입니다! 이전에 드린 정보를 다시 요약해드릴게요. 복합성 피부에 적합한 화장품 추천은 다음과 같습니다:

1. **클렌저**: 젤 타입 클렌저, 예를 들어 세타필 젠틀 스킨 클렌저.
2. **토너**: 알코올 프리 토너, 예를 들어 키엘의 칼렌듈라 허벌 엑스트랙트 토너.
3. **세럼**: 하이알루론산 세럼, 예를 들어 더 오디너리 하이알루론산 2% + B5 세럼.
4. **모이스처라이저**: 오일 프리 젤 크림, 예를 들어 뉴트로지나 하이드로 부스트 워터 젤.
5. **선크림**: 논코메도제닉 선크림, 예를 들어 라로슈포제 안뗄리오스 울트라 라이트 플루이드 SPF50+.
6. **마스크**: 클레이 마스크, 예를 들어 이니스프리 수퍼 화산송이 모공 마스크.

이 제품들은 복합성 피부의 균형을 맞추고 건강하게 유지하는 데 도움을 줄 수 있습니다. 사용 전에는 개인 피부에 맞는지 테스트를 해보는 것이 좋습니다. 더 궁금한 점이 있으면 언제든지 말씀해 주세요!


In [133]:
def conversation_chain(memory, query):
    chat_history = RunnablePassthrough.assign(
        chat_history = RunnableLambda(memory.load_memory_variables) | itemgetter(memory.memory_key)
    )
    llm = ChatOpenAI(model = "gpt-4o", temperature = 0)
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "너는 친절한 화장품 안내 가이드야"),
            MessagesPlaceholder(variable_name = "chat_history"),
            ("human", "{input}")
        ]
    )
    chain = chat_history | prompt | llm | StrOutputParser()
    # StrOutputParser() : 출력된느 값을 자동으로 파싱(불필요한 개행문자, 특수기호 등을 자동으로 처리)
    answer = chain.invoke({"input" : query})
    memory.save_context(inputs = {"human" : query}, outputs = {"ai" : answer})
    # 인풋 아웃풋을 명시적으로 제공하자
    return answer

## 워드 클라우드 시각화

In [ ]:
# pip install wordcloud matplotlib